In [2]:
# packages
import numpy as np
import pandas as pd

In [3]:
# hide warning messages
import warnings
warnings.filterwarnings("ignore")

### DateRange

In [4]:
# getting the daterange in daily frequency to create some dataframes (we're gonna use the marketcap dataset for this)
returns_path = '../../../input/returns/daily.parquet'
returns = pd.read_parquet(returns_path)
daterange = returns.index
daterange

DatetimeIndex(['2005-01-03', '2005-01-04', '2005-01-05', '2005-01-06',
               '2005-01-07', '2005-01-10', '2005-01-11', '2005-01-12',
               '2005-01-13', '2005-01-14',
               ...
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=3773, freq=None)

### Factors

In [5]:
# factors with 10 percentiles
pNYSE_factors = ['pNYSE_size', 'pNYSE_value', 'pNYSE_prof', 'pNYSE_dur', 'pNYSE_valprof', 
                 'pNYSE_nissa', 'pNYSE_accruals', 'pNYSE_growth', 'pNYSE_aturnover', 
                 'pNYSE_gmargins', 'pNYSE_divp', 'pNYSE_ep', 'pNYSE_cfp', 'pNYSE_noa', 
                 'pNYSE_inv', 'pNYSE_invcap', 'pNYSE_igrowth', 'pNYSE_sgrowth', 
                 'pNYSE_lev', 'pNYSE_roaa', 'pNYSE_roea', 'pNYSE_sp', 'pNYSE_gltnoa', 
                 'pNYSE_divg', 'pNYSE_invaci', 'pNYSE_mom', 'pNYSE_indmom', 'pNYSE_valmom',
                 'pNYSE_valmomprof', 'pNYSE_shortint', 'pNYSE_mom12', 'pNYSE_momrev',
                 'pNYSE_lrrev', 'pNYSE_valuem', 'pNYSE_nissm', 'pNYSE_sue', 'pNYSE_roe',
                 'pNYSE_rome', 'pNYSE_roa', 'pNYSE_strev', 'pNYSE_ivol', 'pNYSE_betaarb',
                 'pNYSE_season', 'pNYSE_indrrev', 'pNYSE_indrrevlv', 'pNYSE_indmomrev',
                 'pNYSE_ciss', 'pNYSE_price', 'pNYSE_age', 'pNYSE_shvol']

In [6]:
# factors with just two percentiles (1 and 10)
pNYSE_factors_ = ['pNYSE_fscore', 'pNYSE_debtiss', 'pNYSE_repurch', 'pNYSE_exchsw', 'pNYSE_ipo']

### Functions

drop_ticker function: receives two parameters, percentile and df_returns. 

* percentile is the percentile list (p1, p2, ..., p10).
* df_returns is the returns dataframe of any trade day.

This funtion returns the percentile list with just the ticks that are in the returns dataframe.

In [7]:
def drop_ticker(percentile, date, df_returns):
    drop_tickers = []
    for ticker in percentile:
        if ticker not in df_returns.loc[date][df_returns.loc[date].notna()].index:
            drop_tickers.append(ticker)
    for ticker in drop_tickers:
        percentile.remove(ticker)
    return percentile

portfolio_decile function: receives three parameters, col, df_factors and df_returns.

* col is the percentile NYSE factor column (pNYSE_size, pNYSE_value, ..., pNYSE_ipo)
* df_factors if the factors dataframe of any day.
* df_returns is the returns dataframe of any trade day.

This funtion returns all percentiles (p1, p2, ..., p10), each of them has tickers of firms whose are in this respective percentile and there is its matching column in returns dataframe.

In [8]:
def portfolio_decile(col, date, df_factors, df_returns): 
    # taking out all firms with NA for this specified factor
    temp = df_factors[df_factors[col].notna()]

    # creating an empty list for 10 percentiles
    p1 = []
    p2 = [] 
    p3 = [] 
    p4 = [] 
    p5 = [] 
    p6 = [] 
    p7 = [] 
    p8 = [] 
    p9 = [] 
    p10 = [] 

    # filling the percentile lists with the respective tickers whose are in this percentile
    for permno in temp.index:
        if temp[col][permno] == 1:
            p1.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 2:
            p2.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 3:
            p3.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 4:
            p4.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 5:
            p5.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 6:
            p6.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 7:
            p7.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 8:
            p8.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 9:
            p9.append(temp['TAQ_TICKER'][permno])
        elif temp[col][permno] == 10:
            p10.append(temp['TAQ_TICKER'][permno])

    """
    Now, we need to use the drop_ticker function.
    Thus, we'll have the percentile lists with just the tickers that are in the returns dataframe.
    We'll use a loop to pass for all percentile lists
    """    
    percentiles = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10]
    for p in percentiles:
        drop_ticker(p, date, df_returns)

    return(p1, p2, p3, p4, p5, p6, p7, p8, p9, p10)

value_weight function: receives two parameters, percentile and df_factors.

* percentile is the percentile list (p1, p2, ..., p10).
* df_factors if the factors dataframe of any day.

This funtion returns the value weight of the firms in this percentile (they are in the same sequence of the percentile list).

In [9]:
def value_weight(percentile, df_factors):
    # getting the marketcap value list
    marketcap = list(df_factors[df_factors['TAQ_TICKER'].isin(percentile)]['MARKETCAP'].values)
    # sum of the marketcaps of this percentile firms
    sum_marketcap = sum(marketcap)
    # getting the value weight
    weight = marketcap/sum_marketcap
    return weight

portfolio_formation function: receives four parameters, col, df, df_factors, df_returns.

* col is the percentile NYSE factor column (pNYSE_size, pNYSE_value, ..., pNYSE_shvol).
* df is the factor dataframe which will be filled with the value weighted portfolios.
* df_factors if the factors dataframe of any day.
* df_returns is the returns dataframe of any trade day.

This funtion returns nothing. It just fills the df inputed with portfolio returns.

In [10]:
def portfolio_formation(date, col, df, df_factors, df_returns):
    # getting the percentile lists from portfolio_decile function
    p1, p2, p3, p4, p5, p6, p7, p8, p9, p10 = portfolio_decile(col, date, df_factors, df_returns)

    """
    Now, we'll fill the df (input) dataframe with percentiles portfolios.
    Each percentile column of df will receive the value weighted portfolio of the stock returns in this specified percentile.
    """

    df['p1'][date] = ((returns[p1].loc[date])*(value_weight(p1, df_factors))).sum(axis=0)
    df['p2'][date] = ((returns[p2].loc[date])*(value_weight(p2, df_factors))).sum(axis=0)
    df['p3'][date] = ((returns[p3].loc[date])*(value_weight(p3, df_factors))).sum(axis=0)
    df['p4'][date] = ((returns[p4].loc[date])*(value_weight(p4, df_factors))).sum(axis=0)
    df['p5'][date] = ((returns[p5].loc[date])*(value_weight(p5, df_factors))).sum(axis=0)
    df['p6'][date] = ((returns[p6].loc[date])*(value_weight(p6, df_factors))).sum(axis=0)
    df['p7'][date] = ((returns[p7].loc[date])*(value_weight(p7, df_factors))).sum(axis=0)
    df['p8'][date] = ((returns[p8].loc[date])*(value_weight(p8, df_factors))).sum(axis=0)
    df['p9'][date] = ((returns[p9].loc[date])*(value_weight(p9, df_factors))).sum(axis=0)
    df['p10'][date] = ((returns[p10].loc[date])*(value_weight(p10, df_factors))).sum(axis=0)

portfolio_formation_ function: receives four parameters, col, df, df_factors, df_returns.

* col is the percentile NYSE factor column (pNYSE_fscore, pNYSE_debtiss, pNYSE_repurch, pNYSE_exchsw, pNYSE_ipo).
* df is the factor dataframe which will be filled with the value weighted portfolios.
* df_factors if the factors dataframe of any day.
* df_returns is the returns dataframe of any trade day.

This funtion returns nothing. It just fills the df inputed with portfolio returns.

Obs.: This function do the same thing that portfolio_formation, but it does for binary factors.

In [11]:
def portfolio_formation_(date, col, df, df_factors, df_returns):
    # getting the percentile lists from portfolio_decile function
    p1, p2, p3, p4, p5, p6, p7, p8, p9, p10 = portfolio_decile(col, date, df_factors, df_returns)

    """
    Now, we'll fill the df (input) dataframe with percentiles portfolios.
    Each percentile column of df will receive the value weighted portfolio of the stock returns in this specified percentile.
    """

    df['p1'][date] = ((df_returns[p1].loc[date])*(value_weight(p1, df_factors))).sum(axis=0)
    df['p10'][date] = ((df_returns[p10].loc[date])*(value_weight(p10, df_factors))).sum(axis=0)

n_firms function: receives four parameters, col, df, df_factors, df_returns.

* col is the percentile NYSE factor column (pNYSE_size, pNYSE_value, ..., pNYSE_ipo).
* df is the factor dataframe which will be filled with the value weighted portfolios.
* df_factors if the factors dataframe of any day.
* df_returns is the returns dataframe of any trade day.

This funtion returns nothing. It just fills the df inputed with number of firms.

In [12]:
def n_firms(col, df, date, df_factors, df_returns):
    # getting the percentile lists from portfolio_decile function
    p1, p2, p3, p4, p5, p6, p7, p8, p9, p10 = portfolio_decile(col, date, df_factors, df_returns)

    """
    Now, we'll fill the df (input) dataframe with number of firms by percentile.
    Each percentile column of df will receive the number of firms in this specified percentile.
    """

    df['p1'][date] = len(p1)
    df['p2'][date] = len(p2)
    df['p3'][date] = len(p3)
    df['p4'][date] = len(p4)
    df['p5'][date] = len(p5)
    df['p6'][date] = len(p6)
    df['p7'][date] = len(p7)
    df['p8'][date] = len(p8)
    df['p9'][date] = len(p9)
    df['p10'][date] = len(p10)

### Data Generator Process

#### Portfolio-by-Deciles Loop

In [13]:
pNYSE = pNYSE_factors[0]
pNYSE

'pNYSE_size'

In [14]:
factor = pNYSE[6:]
factor

'size'

In [15]:
p10 = pd.DataFrame(index=daterange, columns=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10'])

# number of firms dataframe
n10 = pd.DataFrame(index=daterange, columns=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10'])

In [16]:
date = daterange[3]
date

Timestamp('2005-01-06 00:00:00')

In [17]:
day = str(date)[:4] + str(date)[5:7] + str(date)[8:10]
day

'20050106'

In [18]:
# factors dataframe
factors_path = f'../../../input/factors/{day}.parquet'
factors = pd.read_parquet(factors_path)
# dropping all firms whose doesn't have TAQ_TICKER, because that is the only variable we can connect to returns database
factors = factors[factors['TAQ_TICKER'] != '<undefined>']
# dropping the firms with NaN value for Market Cap
factors = factors[factors['MARKETCAP'].notna()]
# getting the absolute value of Market Cap
factors['MARKETCAP'] = factors['MARKETCAP'].abs()

In [19]:
# filling the portfolio returns dataframes (intradaily)
portfolio_formation(date, pNYSE, p10, factors, returns) 

In [20]:
# filling the number of firms dataframes (daily)
n_firms(pNYSE, n10, date, factors, returns)

In [21]:
p10

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10
2005-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,0.001774,0.00271,0.002593,0.002261,0.000916,0.001457,0.000238,0.000014,-0.00067,0.000907
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2019-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
n10

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10
2005-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,166,167,193,200,214,241,291,334,587,1972
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2019-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
for pNYSE in pNYSE_factors[1:2]:
    # getting just the factor name (for the output path folder)
    factor = pNYSE[6:]

    # portfolio returns dataframe
    p10 = pd.DataFrame(index=daterange, columns=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10'])

    # number of firms dataframe
    n10 = pd.DataFrame(index=daterange, columns=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10'])
    for date in daterange:
        # we need to convert in the csv's names format
        day = str(date)[:4] + str(date)[5:7] + str(date)[8:10]
        try:
            # factors dataframe
            factors_path = f'../../../input/factors/{day}.parquet'
            factors = pd.read_parquet(factors_path)
            # dropping all firms whose doesn't have TAQ_TICKER, because that is the only variable we can connect to returns database
            factors = factors[factors['TAQ_TICKER'] != '<undefined>']
            # dropping the firms with NaN value for Market Cap
            factors = factors[factors['MARKETCAP'].notna()]
            # getting the absolute value of Market Cap
            factors['MARKETCAP'] = factors['MARKETCAP'].abs()

            # filling the portfolio returns dataframes (intradaily)
            portfolio_formation(date, pNYSE, p10, factors, returns) 
            
            # filling the number of firms dataframes (daily)
            n_firms(pNYSE, n10, date, factors, returns)
        except:
            pass

    # converting portfolio returns dataframes to csv
    output_path = f'../../../output/data/double_check/{factor}/p10.parquet'
    p10.to_parquet(output_path)
    
    # converting number of firms dataframes to csv
    output_path = f'../../../output/data/double_check/{factor}/n10_daily.parquet'
    n10.to_parquet(output_path)

#### Binary Portfolio Loop

In [ ]:
for pNYSE in pNYSE_factors_:
    # getting just the factor name (for the output path folder)
    factor = pNYSE[6:]

    # portfolio returns dataframe
    p10 = pd.DataFrame(index=daterange, columns=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10'])
    
    # number of firms dataframe
    n10 = pd.DataFrame(index=daterange, columns=['p1','p2','p3','p4','p5','p6','p7','p8','p9','p10'])
    for date in daterange:
        # we need to convert in the csv's names format
        day = str(date)[:4] + str(date)[5:7] + str(date)[8:10]
        try:
            # factors dataframe
            factors_path = f'../../../input/factors/{day}.parquet'
            factors = pd.read_parquet(factors_path)
            # dropping all firms whose doesn't have TAQ_TICKER, because that is the only variable we can connect to returns database
            factors = factors[factors['TAQ_TICKER'] != '<undefined>']
            # dropping the firms with NaN value for Market Cap
            factors = factors[factors['MARKETCAP'].notna()]
            # getting the absolute value of Market Cap
            factors['MARKETCAP'] = factors['MARKETCAP'].abs()

            # filling the portfolio returns dataframes (intradaily)
            portfolio_formation(date, pNYSE, p10, factors, returns) 
            
            # filling the number of firms dataframes (daily)
            date = pd.to_datetime(day)
            n_firms(pNYSE, n10, date, factors, returns)
        except:
            pass
    # converting portfolio returns dataframes to csv
    output_path = f'../../../output/data/double_check/{factor}/p10.parquet'
    p10.to_parquet(output_path)

    # converting number of firms dataframes to csv
    output_path = f'../../../output/data/double_check/{factor}/n10_daily.parquet'
    n10.to_parquet(output_path)